In [1]:
import numpy as np
import cvxpy
import scipy
import scs

In [2]:
print(scs.__version__)
assert(int(scs.__version__[0]) >= 2)

2.1.1


In [5]:
# Random LP
np.random.seed(hash('lp') % 2 ** 31)

# Dimensions
n = 100
m = 70

A = scipy.sparse.random(m, n, density=0.2, data_rvs = np.random.randn)
b = np.random.randn(m)
c = np.random.rand(n)

# Problem construction
x = cvxpy.Variable(n)
objective = cvxpy.Minimize(c.T * x)
constraints = [x >= 0, A * x <= b]
prob = cvxpy.Problem(objective, constraints)

prob.solve(solver='SCS', verbose=True, alpha=1)
prob.solve(solver='SCS', verbose=True, alpha=1, acceleration_lookback=0)

----------------------------------------------------------------------------
	SCS v2.1.1 - Splitting Conic Solver
	(c) Brendan O'Donoghue, Stanford University, 2012
----------------------------------------------------------------------------
Lin-sys: sparse-direct, nnz in A = 1500
eps = 1.00e-05, alpha = 1.00, max_iters = 5000, normalize = 1, scale = 1.00
acceleration_lookback = 10, rho_x = 1.00e-03
Variables n = 100, constraints m = 170
Cones:	linear vars: 170
Setup time: 1.30e-02s
----------------------------------------------------------------------------
 Iter | pri res | dua res | rel gap | pri obj | dua obj | kap/tau | time (s)
----------------------------------------------------------------------------
     0| 5.87e+19  5.56e+19  1.00e+00 -6.01e+20  1.88e+20  3.59e+20  1.26e-02 
   100| 2.25e-03  1.16e-02  2.02e-04  5.22e+00  5.22e+00  4.82e-16  1.98e-02 
   200| 1.15e-03  4.15e-03  6.80e-04  5.21e+00  5.22e+00  5.06e-16  2.69e-02 
   300| 3.04e-04  9.36e-04  1.48e-04  5.23e+00 

5.227244629975223

In [4]:
# Optimal control
np.random.seed(hash('opt-control') % 2 ** 31)

# Problem data
T = 10
n, p = (10, 5)
A = np.random.randn(n, n)
B = np.random.randn(n, p)
x_init = np.random.randn(n, 1)
x_final = np.random.randn(n, 1)

def step(A, B, x_prev):
    x = cvxpy.Variable(n, 1)
    u = cvxpy.Variable(p, 1)
    cost = sum(cvxpy.square(u)) + sum(cvxpy.abs(x))
    constraint = (x == A * x_prev + B * u)
    return cost, constraint, x

x = cvxpy.Variable(n, 1)
constraints = [(x == x_init)]
total_cost = 0.
for t in range(T):
    cost_t, constraint_t, x = step(A, B, x)
    constraints.append(constraint_t)
    total_cost += cost_t

constraints.append(x == x_final)
prob = cvxpy.Problem(cvxpy.Minimize(total_cost), constraints)
prob.solve(solver='SCS', verbose=True)
prob.solve(solver='SCS', verbose=True, acceleration_lookback=0)

----------------------------------------------------------------------------
	SCS v2.0.0 - Splitting Conic Solver
	(c) Brendan O'Donoghue, Stanford University, 2012-2017
----------------------------------------------------------------------------
Lin-sys: sparse-indirect, nnz in A = 2170, CG tol ~ 1/iter^(2.00)
eps = 1.00e-05, alpha = 1.50, max_iters = 5000, normalize = 1, scale = 1.00
acceleration_lookback = 20, rho_x = 1.00e-03
Variables n = 310, constraints m = 470
Cones:	primal zero / dual free vars: 120
	linear vars: 200
	soc vars: 150, soc blks: 50
Setup time: 1.51e-02s
----------------------------------------------------------------------------
 Iter | pri res | dua res | rel gap | pri obj | dua obj | kap/tau | time (s)
----------------------------------------------------------------------------
     0| 2.57e+20  2.24e+20  1.00e+00 -1.34e+22  2.26e+21  7.84e+21  1.44e-02 
   100| 2.64e-01  1.85e-01  7.07e-04  4.81e+02  4.81e+02  2.92e-15  3.84e-02 
   200| 4.46e-02  2.05e-02  5.

465.26823692815213

In [5]:
# Lasso
np.random.seed(hash('lasso') % 2 ** 31)

# Dimensions
n = 100
m = 50

x_true = scipy.sparse.rand(n, 1, density=0.1) 
A = scipy.sparse.random(m, n, density=0.2, data_rvs = np.random.randn)
b = A * x_true + 0.1 * np.random.randn(m, 1)
mu = 1

# Problem construction
x = cvxpy.Variable(n)
objective = cvxpy.Minimize(0.5 * cvxpy.sum_squares(A*x - b) + mu * cvxpy.norm1(x))
prob = cvxpy.Problem(objective)

prob.solve(solver='SCS', verbose=True)
prob.solve(solver='SCS', verbose=True, acceleration_lookback=0)

----------------------------------------------------------------------------
	SCS v2.0.0 - Splitting Conic Solver
	(c) Brendan O'Donoghue, Stanford University, 2012-2017
----------------------------------------------------------------------------
Lin-sys: sparse-indirect, nnz in A = 1402, CG tol ~ 1/iter^(2.00)
eps = 1.00e-05, alpha = 1.50, max_iters = 5000, normalize = 1, scale = 1.00
acceleration_lookback = 20, rho_x = 1.00e-03
Variables n = 201, constraints m = 252
Cones:	linear vars: 200
	soc vars: 52, soc blks: 1
Setup time: 1.53e-02s
----------------------------------------------------------------------------
 Iter | pri res | dua res | rel gap | pri obj | dua obj | kap/tau | time (s)
----------------------------------------------------------------------------
     0| 1.25e+20  1.93e+20  1.00e+00 -2.04e+21  8.77e+20  3.64e+20  1.32e-02 
    80| 1.03e-06  4.19e-06  3.67e-07  5.33e+00  5.33e+00  1.59e-15  2.44e-02 
-------------------------------------------------------------------

5.334218831597699

In [6]:
# Nonnegative Lasso
np.random.seed(hash('nonneg-lasso') % 2 ** 31)

# Dimensions
n = 100
m = 50

x_true = scipy.sparse.rand(n, 1, density=0.1) 
A = scipy.sparse.random(m, n, density=0.2, data_rvs = np.random.randn)
b = A * x_true + 0.1 * np.random.randn(m, 1)
mu = 1

# Problem construction
x = cvxpy.Variable(n)
objective = cvxpy.Minimize(0.5 * cvxpy.sum_squares(A*x - b) + mu * cvxpy.norm1(x))
constraints = [x >= 0]
prob = cvxpy.Problem(objective, constraints)

prob.solve(solver='SCS', verbose=True)
prob.solve(solver='SCS', verbose=True, acceleration_lookback=0)

----------------------------------------------------------------------------
	SCS v2.0.0 - Splitting Conic Solver
	(c) Brendan O'Donoghue, Stanford University, 2012-2017
----------------------------------------------------------------------------
Lin-sys: sparse-indirect, nnz in A = 1502, CG tol ~ 1/iter^(2.00)
eps = 1.00e-05, alpha = 1.50, max_iters = 5000, normalize = 1, scale = 1.00
acceleration_lookback = 20, rho_x = 1.00e-03
Variables n = 201, constraints m = 352
Cones:	linear vars: 300
	soc vars: 52, soc blks: 1
Setup time: 1.47e-02s
----------------------------------------------------------------------------
 Iter | pri res | dua res | rel gap | pri obj | dua obj | kap/tau | time (s)
----------------------------------------------------------------------------
     0| 1.42e+20  2.20e+20  1.00e+00 -1.70e+21  1.39e+21  2.99e+20  1.40e-02 
    80| 2.12e-07  1.07e-06  2.64e-08  5.18e+00  5.18e+00  2.55e-16  3.24e-02 
-------------------------------------------------------------------

5.183071594079891

In [7]:
# SDP for closest elemwise-positive PSD matrix in some metric
np.random.seed(hash('sdp') % 2 ** 31)

# create data P
n = 50
P = np.random.randn(n, n)
P = P + P.T
Z = cvxpy.semidefinite(n)

objective = cvxpy.Maximize(cvxpy.lambda_min(P - Z))
prob = cvxpy.Problem(objective, [Z >= 0])
prob.solve(solver='SCS', verbose=True)
prob.solve(solver='SCS', verbose=True, acceleration_lookback=0)

----------------------------------------------------------------------------
	SCS v2.0.0 - Splitting Conic Solver
	(c) Brendan O'Donoghue, Stanford University, 2012-2017
----------------------------------------------------------------------------
Lin-sys: sparse-indirect, nnz in A = 6325, CG tol ~ 1/iter^(2.00)
eps = 1.00e-05, alpha = 1.50, max_iters = 5000, normalize = 1, scale = 1.00
acceleration_lookback = 20, rho_x = 1.00e-03
Variables n = 1276, constraints m = 6275
Cones:	primal zero / dual free vars: 1225
	linear vars: 2500
	sd vars: 2550, sd blks: 2
Setup time: 1.53e-02s
----------------------------------------------------------------------------
 Iter | pri res | dua res | rel gap | pri obj | dua obj | kap/tau | time (s)
----------------------------------------------------------------------------
     0| 3.96e+01  4.31e+01  9.99e-01 -2.38e+02  8.35e+02  1.91e-13  3.67e-02 
   100| 4.49e-05  1.21e-05  4.84e-06  1.78e+01  1.78e+01  5.90e-15  2.56e-01 
   120| 9.66e-06  2.97e-06  

-17.814508069152765

In [8]:
# Basis pursuit
np.random.seed(hash('basis-pursuit') % 2 ** 31)

n = 300
m = 100
x = cvxpy.Variable(n)
A = np.random.rand(m, n)
x0 = scipy.sparse.rand(n, 1, 0.1)
b = A*x0

prob = cvxpy.Problem(cvxpy.Minimize(cvxpy.norm1(x)), [A*x == b])
prob.solve(solver='SCS', verbose=True)
prob.solve(solver='SCS', verbose=True, acceleration_lookback=0)

----------------------------------------------------------------------------
	SCS v2.0.0 - Splitting Conic Solver
	(c) Brendan O'Donoghue, Stanford University, 2012-2017
----------------------------------------------------------------------------
Lin-sys: sparse-indirect, nnz in A = 31200, CG tol ~ 1/iter^(2.00)
eps = 1.00e-05, alpha = 1.50, max_iters = 5000, normalize = 1, scale = 1.00
acceleration_lookback = 20, rho_x = 1.00e-03
Variables n = 600, constraints m = 700
Cones:	primal zero / dual free vars: 100
	linear vars: 600
Setup time: 1.56e-02s
----------------------------------------------------------------------------
 Iter | pri res | dua res | rel gap | pri obj | dua obj | kap/tau | time (s)
----------------------------------------------------------------------------
     0| 9.69e+00  1.45e+01  9.99e-01 -1.24e+03  4.85e+01  1.52e-13  2.07e-02 
   100| 1.79e-04  2.91e-04  6.95e-05  1.39e+01  1.39e+01  8.90e-15  1.16e-01 
   140| 6.17e-06  7.62e-06  9.54e-06  1.39e+01  1.39e+01  

13.868068778792175

In [9]:
# Chebyshev
np.random.seed(hash('chebyshev') % 2 ** 31)

def normalized_data_matrix(m, n):
    A = np.random.randn(m, n)
    A /= np.sqrt(np.sum(A**2, 0))
    return A

m = 100
n = 200
k = 50
A = [normalized_data_matrix(m, n) for i in range(k)]
B = normalized_data_matrix(k, n)
c = np.random.rand(k)

x = cvxpy.Variable(n)
t = cvxpy.Variable(k)

f = cvxpy.max_entries(t + cvxpy.abs(B * x - c))
constraints = []
for i in range(k):
    constraints.append(cvxpy.norm(A[i]*x) <= t[i])

prob = cvxpy.Problem(cvxpy.Minimize(f), constraints)
prob.solve(solver='SCS', verbose=True)
prob.solve(solver='SCS', verbose=True, acceleration_lookback=0)

----------------------------------------------------------------------------
	SCS v2.0.0 - Splitting Conic Solver
	(c) Brendan O'Donoghue, Stanford University, 2012-2017
----------------------------------------------------------------------------
Lin-sys: sparse-indirect, nnz in A = 1020400, CG tol ~ 1/iter^(2.00)
eps = 1.00e-05, alpha = 1.50, max_iters = 5000, normalize = 1, scale = 1.00
acceleration_lookback = 20, rho_x = 1.00e-03
Variables n = 351, constraints m = 5250
Cones:	linear vars: 200
	soc vars: 5050, soc blks: 50
Setup time: 4.50e-02s
----------------------------------------------------------------------------
 Iter | pri res | dua res | rel gap | pri obj | dua obj | kap/tau | time (s)
----------------------------------------------------------------------------
     0| 4.42e+20  4.17e+20  1.00e+00 -2.33e+20  1.23e+21  1.15e+20  6.19e-02 
   100| 1.74e-03  4.07e-03  5.03e-04  9.60e-01  9.58e-01  5.30e-16  2.15e+00 
   200| 7.39e-06  1.95e-05  5.22e-07  9.59e-01  9.59e-01  1.

0.9594332606540568

In [10]:
# Least absolute deviation
np.random.seed(hash('least-abs-dev') % 2 ** 31)

m = 5000
n = 200

A = np.random.randn(m,n);
b = A.dot(np.random.randn(n))

# Add outlier noise.
k = int(0.02 * m)
idx = np.random.randint(m, size=k)
b[idx] += 10 * np.random.randn(k)

x = cvxpy.Variable(n)

prob = cvxpy.Problem(cvxpy.Minimize(cvxpy.norm1(A*x - b)))
prob.solve(solver='SCS', verbose=True)
prob.solve(solver='SCS', verbose=True, acceleration_lookback=0)

----------------------------------------------------------------------------
	SCS v2.0.0 - Splitting Conic Solver
	(c) Brendan O'Donoghue, Stanford University, 2012-2017
----------------------------------------------------------------------------
Lin-sys: sparse-indirect, nnz in A = 2010000, CG tol ~ 1/iter^(2.00)
eps = 1.00e-05, alpha = 1.50, max_iters = 5000, normalize = 1, scale = 1.00
acceleration_lookback = 20, rho_x = 1.00e-03
Variables n = 5200, constraints m = 10000
Cones:	linear vars: 10000
Setup time: 7.98e-02s
----------------------------------------------------------------------------
 Iter | pri res | dua res | rel gap | pri obj | dua obj | kap/tau | time (s)
----------------------------------------------------------------------------
     0| 2.76e+00  8.14e+00  1.00e+00 -1.72e+05  4.65e+03  3.95e-11  1.25e-01 
   100| 4.91e-05  2.39e-03  2.00e-05  7.49e+02  7.49e+02  6.65e-11  2.47e+00 
   140| 1.04e-07  3.91e-06  4.86e-07  7.49e+02  7.49e+02  3.78e-11  3.21e+00 
--------

748.629441875814

In [11]:
# P-norm
np.random.seed(hash('p-norm') % 2 ** 31)

n = 20
m = int(n / 4)

G = np.random.randn(m, n)
f = np.random.randn(m, 1)

power = np.pi
x = cvxpy.Variable(n)
constraints = [G*x == f]
prob = cvxpy.Problem(cvxpy.Minimize(cvxpy.norm(x, power)), constraints)

prob.solve(solver='SCS', verbose=True)
prob.solve(solver='SCS', verbose=True, acceleration_lookback=0)

----------------------------------------------------------------------------
	SCS v2.0.0 - Splitting Conic Solver
	(c) Brendan O'Donoghue, Stanford University, 2012-2017
----------------------------------------------------------------------------
Lin-sys: sparse-indirect, nnz in A = 1301, CG tol ~ 1/iter^(2.00)
eps = 1.00e-05, alpha = 1.50, max_iters = 5000, normalize = 1, scale = 1.00
acceleration_lookback = 20, rho_x = 1.00e-03
Variables n = 261, constraints m = 706
Cones:	primal zero / dual free vars: 6
	linear vars: 40
	soc vars: 660, soc blks: 220
Setup time: 1.47e-02s
----------------------------------------------------------------------------
 Iter | pri res | dua res | rel gap | pri obj | dua obj | kap/tau | time (s)
----------------------------------------------------------------------------
     0| 3.10e+00  6.93e+00  5.92e-01 -4.18e-01  1.03e+00  0.00e+00  1.71e-02 
   100| 2.74e-03  1.41e-03  3.57e-05  3.75e-01  3.75e-01  9.56e-19  3.48e-02 
   200| 2.98e-04  1.30e-04  1.38

0.374797039226272

In [12]:
# L1-regularized Logistic regression
np.random.seed(hash('log-reg') % 2 ** 31)

p = 1000   # features
q = 10 * p  # total samples

w_true = np.random.randn(p, 1)
X_tmp = np.random.randn(p, q)

ips = -w_true.T.dot(X_tmp)
ps = (np.exp(ips)/(1 + np.exp(ips))).T
labels = 2*(np.random.rand(q,1) < ps) - 1

X_pos = X_tmp[:,np.where(labels==1)[0]]
X_neg = X_tmp[:,np.where(labels==-1)[0]]
X = np.hstack([X_pos, -X_neg]) # include labels with data

lam = 2
w = cvxpy.Variable(p, 1)
obj = (cvxpy.sum_entries(cvxpy.log_sum_exp(cvxpy.vstack([np.zeros((1,q)), w.T*X]), axis = 0))
        + lam * cvxpy.norm(w,1))

prob = cvxpy.Problem(cvxpy.Minimize(obj))
prob.solve(solver='SCS', verbose=True)
prob.solve(solver='SCS', verbose=True, acceleration_lookback=0)

----------------------------------------------------------------------------
	SCS v2.0.0 - Splitting Conic Solver
	(c) Brendan O'Donoghue, Stanford University, 2012-2017
----------------------------------------------------------------------------
Lin-sys: sparse-indirect, nnz in A = 10064000, CG tol ~ 1/iter^(2.00)
eps = 1.00e-05, alpha = 1.50, max_iters = 5000, normalize = 1, scale = 1.00
acceleration_lookback = 20, rho_x = 1.00e-03
Variables n = 32000, constraints m = 72000
Cones:	linear vars: 12000
	exp vars: 60000, dual exp vars: 0
Setup time: 4.22e-01s
----------------------------------------------------------------------------
 Iter | pri res | dua res | rel gap | pri obj | dua obj | kap/tau | time (s)
----------------------------------------------------------------------------
     0| 2.93e+21  4.84e+21  1.00e+00 -1.74e+25  3.88e+25  5.91e+24  9.02e-01 
   100| 3.52e-02  3.25e-02  1.45e-02  9.41e+02  9.69e+02  2.52e-12  4.63e+01 
   200| 3.29e-03  3.67e-03  2.51e-04  9.59e+02  9

958.3582765262744

In [13]:
# Matrix completion
np.random.seed(hash('matrix-completion') % 2 ** 31)

m = 100
n = 50

M = np.random.randn(m * n)
n_missing = int(0.8 * m * n)
missing_idx = np.random.permutation(m * n)[:n_missing]
valid_idx = np.setdiff1d(np.arange(m * n), missing_idx)

M[missing_idx] = 0.
X = cvxpy.Variable(m * n)

lam = 0.5
diff = cvxpy.reshape(X, m, n) - np.reshape(M, (m, n))
obj = cvxpy.norm(diff, "nuc") + lam * cvxpy.sum_squares(X)
constraints = [X[valid_idx] == M[valid_idx]]

prob = cvxpy.Problem(cvxpy.Minimize(obj), constraints)

prob.solve(solver='SCS', verbose=True)
prob.solve(solver='SCS', verbose=True, acceleration_lookback=0)

----------------------------------------------------------------------------
	SCS v2.0.0 - Splitting Conic Solver
	(c) Brendan O'Donoghue, Stanford University, 2012-2017
----------------------------------------------------------------------------
Lin-sys: sparse-indirect, nnz in A = 49677, CG tol ~ 1/iter^(2.00)
eps = 1.00e-05, alpha = 1.50, max_iters = 5000, normalize = 1, scale = 1.00
acceleration_lookback = 20, rho_x = 1.00e-03
Variables n = 27501, constraints m = 33502
Cones:	primal zero / dual free vars: 17175
	soc vars: 5002, soc blks: 1
	sd vars: 11325, sd blks: 1
Setup time: 2.08e-02s
----------------------------------------------------------------------------
 Iter | pri res | dua res | rel gap | pri obj | dua obj | kap/tau | time (s)
----------------------------------------------------------------------------
     0| 2.11e+21  2.08e+21  1.00e+00 -4.73e+23  4.24e+23  1.25e+23  1.04e-01 
   100| 1.96e-05  1.64e-05  2.96e-07  8.24e+02  8.24e+02  1.62e-14  2.51e+00 
   120| 2.89e

824.0789125556516

In [14]:
# Min-norm
np.random.seed(hash('min-norm') % 2 ** 31)

m = 500
n = int(m / 2)
A = np.random.randn(m, n)
b = 10 * np.random.randn(m, 1)
G = 2 * np.random.randn(2 * n, n)

x = cvxpy.Variable(n)
obj = cvxpy.norm(A * x - b)
constraints = [cvxpy.norm(G * x) <= 1]

prob = cvxpy.Problem(cvxpy.Minimize(obj), constraints)
prob.solve(solver='SCS', verbose=True)
prob.solve(solver='SCS', verbose=True, acceleration_lookback=0)

----------------------------------------------------------------------------
	SCS v2.0.0 - Splitting Conic Solver
	(c) Brendan O'Donoghue, Stanford University, 2012-2017
----------------------------------------------------------------------------
Lin-sys: sparse-indirect, nnz in A = 250003, CG tol ~ 1/iter^(2.00)
eps = 1.00e-05, alpha = 1.50, max_iters = 5000, normalize = 1, scale = 1.00
acceleration_lookback = 20, rho_x = 1.00e-03
Variables n = 252, constraints m = 1003
Cones:	linear vars: 1
	soc vars: 1002, soc blks: 2
Setup time: 1.51e-02s
----------------------------------------------------------------------------
 Iter | pri res | dua res | rel gap | pri obj | dua obj | kap/tau | time (s)
----------------------------------------------------------------------------
     0| 2.54e+20  1.04e+21  1.00e+00 -2.94e+22  1.04e+23  9.42e+20  1.97e-02 
    60| 2.78e-07  5.41e-06  2.84e-08  2.25e+02  2.25e+02  2.89e-14  1.61e-01 
----------------------------------------------------------------

225.22552672016766

In [15]:
# Lyapunov stability
np.random.seed(hash('lyapunov') % 2 ** 31)

n = 100
A = np.diag(-np.logspace(-0.5, 1, n))
U = scipy.linalg.orth(np.random.randn(n,n))
A = U.T.dot(A.dot(U))

P = cvxpy.Symmetric(n, n)
obj = cvxpy.trace(P)
constraints = [A.T*P + P*A << -np.eye(n), P >> np.eye(n)]

prob = cvxpy.Problem(cvxpy.Minimize(obj), constraints)
prob.solve(solver='SCS', verbose=True)
prob.solve(solver='SCS', verbose=True, acceleration_lookback=0)

----------------------------------------------------------------------------
	SCS v2.0.0 - Splitting Conic Solver
	(c) Brendan O'Donoghue, Stanford University, 2012-2017
----------------------------------------------------------------------------
Lin-sys: sparse-indirect, nnz in A = 1000100, CG tol ~ 1/iter^(2.00)
eps = 1.00e-05, alpha = 1.50, max_iters = 5000, normalize = 1, scale = 1.00
acceleration_lookback = 20, rho_x = 1.00e-03
Variables n = 5050, constraints m = 10100
Cones:	sd vars: 10100, sd blks: 2
Setup time: 4.58e-02s
----------------------------------------------------------------------------
 Iter | pri res | dua res | rel gap | pri obj | dua obj | kap/tau | time (s)
----------------------------------------------------------------------------
     0| 2.14e+20  1.29e+21  9.71e-01  1.99e+21  1.36e+23  7.89e+19  8.83e-02 
   100| 2.90e-02  1.76e-02  5.15e-04  1.04e+02  1.03e+02  2.14e-14  3.65e+00 
   200| 6.32e-03  4.57e-03  3.41e-05  1.04e+02  1.04e+02  8.97e-15  7.52e+00 


103.81819066818625

In [16]:
# Portfolio
np.random.seed(hash('portfolio') % 2 ** 31)

m = 500
n = 5000
density = 0.1

mu = np.exp(0.01 * np.random.randn(n)) - 1.  # returns
D = np.random.rand(n) / 10.  # idiosyncratic risk
F = scipy.sparse.rand(n, m, density) / 10.  # factor model

lambda_risk = 1
leverage = 1

x = cvxpy.Variable(n)
obj = mu.T * x - lambda_risk * (cvxpy.sum_squares(F.T.dot(x)) +
                              cvxpy.sum_squares(cvxpy.mul_elemwise(D, x)))

constraints = [cvxpy.sum_entries(x) == leverage, x >= 0]

prob = cvxpy.Problem(cvxpy.Maximize(obj), constraints)
prob.solve(solver='SCS', verbose=True)
prob.solve(solver='SCS', verbose=True, acceleration_lookback=0)

----------------------------------------------------------------------------
	SCS v2.0.0 - Splitting Conic Solver
	(c) Brendan O'Donoghue, Stanford University, 2012-2017
----------------------------------------------------------------------------
Lin-sys: sparse-indirect, nnz in A = 265004, CG tol ~ 1/iter^(2.00)
eps = 1.00e-05, alpha = 1.50, max_iters = 5000, normalize = 1, scale = 1.00
acceleration_lookback = 20, rho_x = 1.00e-03
Variables n = 5002, constraints m = 10505
Cones:	primal zero / dual free vars: 1
	linear vars: 5000
	soc vars: 5504, soc blks: 2
Setup time: 1.83e-02s
----------------------------------------------------------------------------
 Iter | pri res | dua res | rel gap | pri obj | dua obj | kap/tau | time (s)
----------------------------------------------------------------------------
     0| 5.20e+21  6.68e+20  9.99e-01 -1.27e+22 -8.10e+18  9.50e+21  2.24e-02 
   100| 2.05e-03  2.57e-04  4.14e-04 -1.57e-02 -1.53e-02  3.94e-15  8.78e-01 
   200| 2.36e-04  4.50e-06

0.01526880985319972

In [17]:
# Sparse covariance estimation
np.random.seed(hash('cov-estim') % 2 ** 31)

num_samples = 10
n = 20
lam = 0.1

A = scipy.sparse.rand(n, n, 0.01)
A = A.T.dot(A).todense() + 0.1 * np.eye(n)
L = np.linalg.cholesky(np.linalg.inv(A))
X = L.dot(np.random.randn(n, num_samples)) # Draw m experiments according to the covariance matrix A^-1
S = X.dot(X.T) / num_samples # Estimate of covariance matrix
mask = np.ones((n,n)) - np.eye(n)

theta = cvxpy.Variable(n, n)

obj = (lam*cvxpy.norm1(cvxpy.mul_elemwise(mask, theta)) +
       cvxpy.trace(S * theta) - cvxpy.log_det(theta))
prob = cvxpy.Problem(cvxpy.Minimize(obj))

prob.solve(solver='SCS', verbose=True)
prob.solve(solver='SCS', verbose=True, acceleration_lookback=0)

----------------------------------------------------------------------------
	SCS v2.0.0 - Splitting Conic Solver
	(c) Brendan O'Donoghue, Stanford University, 2012-2017
----------------------------------------------------------------------------
Lin-sys: sparse-indirect, nnz in A = 5260, CG tol ~ 1/iter^(2.00)
eps = 1.00e-05, alpha = 1.50, max_iters = 5000, normalize = 1, scale = 1.00
acceleration_lookback = 20, rho_x = 1.00e-03
Variables n = 2060, constraints m = 3490
Cones:	primal zero / dual free vars: 1600
	linear vars: 800
	sd vars: 1030, sd blks: 2
	exp vars: 60, dual exp vars: 0
Setup time: 1.55e-02s
----------------------------------------------------------------------------
 Iter | pri res | dua res | rel gap | pri obj | dua obj | kap/tau | time (s)
----------------------------------------------------------------------------
     0| 6.50e+20  6.67e+20  1.00e+00 -1.68e+23  1.28e+23  1.26e+23  4.56e-02 
   100| 1.50e-02  8.44e-03  3.59e-04  3.32e+01  3.32e+01  1.77e-13  5.05e-0

33.154008669789135

In [18]:
# Fused Lasso
np.random.seed(hash('fused-lasso') % 2 ** 31)

m = 1000
ni = 10
k = 1000
rho=0.05
sigma=0.05

A = np.random.randn(m, ni * k)
A /= np.sqrt(np.sum(A ** 2, 0))

x0 = np.zeros(ni * k)
for i in range(k):
    if np.random.rand() < rho:
        x0[i * ni:(i + 1) * ni] = np.random.rand()

b = A.dot(x0) + sigma * np.random.randn(m)
lam = 0.1 * sigma * np.sqrt(m * np.log(ni * k))

x = cvxpy.Variable(ni * k)
obj = cvxpy.sum_squares(A * x - b) + lam * cvxpy.norm1(x) + lam * cvxpy.tv(x)
prob = cvxpy.Problem(cvxpy.Minimize(obj))

prob.solve(solver='SCS', verbose=True)
prob.solve(solver='SCS', verbose=True, acceleration_lookback=0)

----------------------------------------------------------------------------
	SCS v2.0.0 - Splitting Conic Solver
	(c) Brendan O'Donoghue, Stanford University, 2012-2017
----------------------------------------------------------------------------
Lin-sys: sparse-indirect, nnz in A = 10099996, CG tol ~ 1/iter^(2.00)
eps = 1.00e-05, alpha = 1.50, max_iters = 5000, normalize = 1, scale = 1.00
acceleration_lookback = 20, rho_x = 1.00e-03
Variables n = 30000, constraints m = 41000
Cones:	linear vars: 39998
	soc vars: 1002, soc blks: 1
Setup time: 1.49e+00s
----------------------------------------------------------------------------
 Iter | pri res | dua res | rel gap | pri obj | dua obj | kap/tau | time (s)
----------------------------------------------------------------------------
     0| 1.58e+21  2.61e+21  1.00e+00 -4.58e+23  2.89e+23  2.05e+23  6.36e-01 
   100| 1.20e-03  5.59e-03  1.10e-05  1.14e+02  1.14e+02  3.43e-14  2.56e+01 
   200| 4.82e-06  2.33e-05  4.88e-07  1.14e+02  1.14e+0

114.23726216498757

In [19]:
# SVM
np.random.seed(hash('svm') % 2 ** 31)

m = 150
n = 500
A = np.random.randn(m, n)
x0 = np.random.rand(n)
y = np.sign(A.dot(x0) + 0.05*np.random.randn(m))

lam = 1.0
x = cvxpy.Variable(n)
obj = ((1./m) * cvxpy.sum_entries(cvxpy.pos(1 - cvxpy.mul_elemwise(y, A * x)))
       + lam * cvxpy.norm(x, 1))

prob = cvxpy.Problem(cvxpy.Minimize(obj))
prob.solve(solver='SCS', verbose=True)
prob.solve(solver='SCS', verbose=True, acceleration_lookback=0)

----------------------------------------------------------------------------
	SCS v2.0.0 - Splitting Conic Solver
	(c) Brendan O'Donoghue, Stanford University, 2012-2017
----------------------------------------------------------------------------
Lin-sys: sparse-indirect, nnz in A = 77300, CG tol ~ 1/iter^(2.00)
eps = 1.00e-05, alpha = 1.50, max_iters = 5000, normalize = 1, scale = 1.00
acceleration_lookback = 20, rho_x = 1.00e-03
Variables n = 1150, constraints m = 1300
Cones:	linear vars: 1300
Setup time: 1.10e-02s
----------------------------------------------------------------------------
 Iter | pri res | dua res | rel gap | pri obj | dua obj | kap/tau | time (s)
----------------------------------------------------------------------------
     0| 1.44e+20  4.24e+20  1.00e+00 -3.38e+21  2.83e+21  1.05e+21  1.36e-02 
   100| 8.96e-02  1.98e-01  8.02e-02  3.02e+00  3.63e+00  1.51e-15  1.54e-01 
   200| 4.36e-02  7.37e-02  4.11e-02  1.93e+00  1.74e+00  1.67e-16  3.32e-01 
   300| 4.07

1.0172207214435558

In [20]:
# Robust PCA
np.random.seed(hash('robust-pca') % 2 ** 31)

n = 100
r = 10 # Rank
density = 0.1

L0 = np.random.randn(n, r).dot(np.random.randn(r, n)) # Low rank matrix
S0 = scipy.sparse.rand(n, n, density) # Sparse matrix w/ Normally distributed entries.
S0.data = 10 * np.random.randn(len(S0.data))

M = L0 + S0

L = cvxpy.Variable(n, n)
S = cvxpy.Variable(n, n)

lam = 0.1
obj = cvxpy.norm(L, "nuc") + lam * cvxpy.norm1(S)
constraints = [L + S == M]

prob = cvxpy.Problem(cvxpy.Minimize(obj), constraints)
prob.solve(solver='SCS', verbose=True)
prob.solve(solver='SCS', verbose=True, acceleration_lookback=0)

----------------------------------------------------------------------------
	SCS v2.0.0 - Splitting Conic Solver
	(c) Brendan O'Donoghue, Stanford University, 2012-2017
----------------------------------------------------------------------------
Lin-sys: sparse-indirect, nnz in A = 139900, CG tol ~ 1/iter^(2.00)
eps = 1.00e-05, alpha = 1.50, max_iters = 5000, normalize = 1, scale = 1.00
acceleration_lookback = 20, rho_x = 1.00e-03
Variables n = 70000, constraints m = 80000
Cones:	primal zero / dual free vars: 39900
	linear vars: 20000
	sd vars: 20100, sd blks: 1
Setup time: 1.82e-02s
----------------------------------------------------------------------------
 Iter | pri res | dua res | rel gap | pri obj | dua obj | kap/tau | time (s)
----------------------------------------------------------------------------
     0| 3.28e+21  3.59e+21  1.00e+00 -1.10e+25  4.78e+24  1.04e+24  6.82e-02 
   100| 7.81e-04  9.05e-04  1.63e-04  1.80e+03  1.80e+03  5.77e-14  5.68e+00 
   200| 1.77e-04  1.7

1801.4294902487838

In [21]:
# Robust Gaussian covariance estimation
np.random.seed(hash('rob-cov-var') % 2 ** 31)

m = 11 # Number of observations of each random variable
n = 5 # Number of random variables
k = 3 # Needs to be less than m. 

A = np.matrix(np.random.rand(m, n))
A -= np.mean(A, axis=0)
K = np.array([(A[i].T * A[i]).flatten() for i in range(m)])

sigma_inv1 = cvxpy.Variable(n,n) # Inverse covariance matrix
t = cvxpy.Variable(m)
tdet = cvxpy.Variable(1)

obj = cvxpy.sum_largest(t+tdet, k)
z = K*cvxpy.reshape(sigma_inv1, n*n, 1)
constraints = [-cvxpy.log_det(sigma_inv1) <= tdet, t == z]
prob = cvxpy.Problem(cvxpy.Minimize(obj), constraints)
prob.solve(solver='SCS', verbose=True)
prob.solve(solver='SCS', verbose=True, acceleration_lookback=0)

----------------------------------------------------------------------------
	SCS v2.0.0 - Splitting Conic Solver
	(c) Brendan O'Donoghue, Stanford University, 2012-2017
----------------------------------------------------------------------------
Lin-sys: sparse-indirect, nnz in A = 597, CG tol ~ 1/iter^(2.00)
eps = 1.00e-05, alpha = 1.50, max_iters = 5000, normalize = 1, scale = 1.00
acceleration_lookback = 20, rho_x = 1.00e-03
Variables n = 139, constraints m = 219
Cones:	primal zero / dual free vars: 111
	linear vars: 23
	sd vars: 70, sd blks: 2
	exp vars: 15, dual exp vars: 0
Setup time: 9.31e-03s
----------------------------------------------------------------------------
 Iter | pri res | dua res | rel gap | pri obj | dua obj | kap/tau | time (s)
----------------------------------------------------------------------------
     0| 1.87e+20  7.34e+19  1.00e+00 -2.69e+21  5.60e+20  2.02e+21  1.42e-02 
   100| 1.84e-01  8.31e-02  1.32e-03 -2.54e+01 -2.53e+01  6.08e-15  4.38e-02 
   2

-26.469884439519348

In [22]:
# Infinite push
np.random.seed(hash('infinite-push') % 2 ** 31)

m = 100
n = 200
d = 20
np.random.seed(0)

Xp = np.random.randn(m, d)
Xn = np.random.randn(n, d)

lam = 1
theta = cvxpy.Variable(d)
Z = cvxpy.max_elemwise(1 - (Xp * theta * np.ones((1,n)) - (Xn * theta * np.ones((1,m))).T), 0)
obj = cvxpy.max_entries(cvxpy.sum_entries(Z, axis=0)) + lam * cvxpy.sum_squares(theta)
prob = cvxpy.Problem(cvxpy.Minimize(obj))

prob.solve(solver='SCS', verbose=True)
prob.solve(solver='SCS', verbose=True, acceleration_lookback=0)

----------------------------------------------------------------------------
	SCS v2.0.0 - Splitting Conic Solver
	(c) Brendan O'Donoghue, Stanford University, 2012-2017
----------------------------------------------------------------------------
Lin-sys: sparse-indirect, nnz in A = 460222, CG tol ~ 1/iter^(2.00)
eps = 1.00e-05, alpha = 1.50, max_iters = 5000, normalize = 1, scale = 1.00
acceleration_lookback = 20, rho_x = 1.00e-03
Variables n = 20022, constraints m = 40222
Cones:	linear vars: 40200
	soc vars: 22, soc blks: 1
Setup time: 3.20e-02s
----------------------------------------------------------------------------
 Iter | pri res | dua res | rel gap | pri obj | dua obj | kap/tau | time (s)
----------------------------------------------------------------------------
     0| 9.15e+20  1.57e+21  1.00e+00 -3.46e+22  1.28e+23  1.75e+22  3.11e-01 
   100| 4.66e-03  1.41e-02  6.07e-04  9.99e+01  9.98e+01  7.30e-15  2.81e+00 
   200| 2.48e-06  1.99e-05  1.40e-07  1.00e+02  1.00e+02  9

100.00010722786132

In [23]:
# Quantile regression
np.random.seed(hash('quantile-regression') % 2 ** 31)

m = 100 # Number of data entries
n = 5 # Number of weights
k = 20 # Number of quantiles
p = 1
sigma = 0.1

x = np.random.rand(m)* 2 * np.pi * p
y = np.sin(x) + sigma * np.sin(x) * np.random.randn(m)
alphas = np.linspace(1. / (k + 1), 1 - 1. / (k + 1), k) # Do a bunch of quantiles at once

# RBF (Radial Basis Function) features
mu_rbf = np.array([np.linspace(-1, 2 * np.pi * p + 1, n)])
mu_sig = (2 * np.pi * p + 2)/n
X = np.exp(-(mu_rbf.T - x).T**2 / (2 * mu_sig**2)) # Gaussian

theta = cvxpy.Variable(n,k)

def quantile_loss(alphas, theta, X, y):
    m, n = X.shape
    k = len(alphas)
    Y = np.tile(y.flatten(), (k, 1)).T
    A = np.tile(alphas, (m, 1))
    Z = X * theta - Y
    return cvxpy.sum_entries(
        cvxpy.max_elemwise(
            cvxpy.mul_elemwise(-A, Z),
            cvxpy.mul_elemwise(1 - A, Z)))

obj = quantile_loss(alphas, theta, X, y)
constraints = [X*(theta[:,1:] - theta[:,:-1]) >= 0]
prob = cvxpy.Problem(cvxpy.Minimize(obj), constraints)

prob.solve(solver='SCS', verbose=True)
prob.solve(solver='SCS', verbose=True, acceleration_lookback=0)

----------------------------------------------------------------------------
	SCS v2.0.0 - Splitting Conic Solver
	(c) Brendan O'Donoghue, Stanford University, 2012-2017
----------------------------------------------------------------------------
Lin-sys: sparse-indirect, nnz in A = 43000, CG tol ~ 1/iter^(2.00)
eps = 1.00e-05, alpha = 1.50, max_iters = 5000, normalize = 1, scale = 1.00
acceleration_lookback = 20, rho_x = 1.00e-03
Variables n = 2100, constraints m = 5900
Cones:	linear vars: 5900
Setup time: 1.24e-02s
----------------------------------------------------------------------------
 Iter | pri res | dua res | rel gap | pri obj | dua obj | kap/tau | time (s)
----------------------------------------------------------------------------
     0| 1.28e+00  1.97e+00  9.99e-01 -8.82e+02  7.71e+01  3.39e-13  8.75e-02 
   100| 1.54e-03  3.46e-03  2.81e-03  3.76e+01  3.74e+01  6.04e-14  8.00e-01 
   200| 2.11e-03  4.95e-03  6.27e-04  3.75e+01  3.75e+01  7.79e-13  1.55e+00 
   300| 1.05

  3700| 9.10e-06  9.33e-05  1.27e-05  3.75e+01  3.75e+01  5.41e-14  5.03e+01 
  3800| 8.31e-06  1.00e-04  1.77e-05  3.75e+01  3.75e+01  5.41e-14  5.29e+01 
  3900| 8.33e-06  9.86e-05  1.66e-05  3.75e+01  3.75e+01  3.76e-14  5.47e+01 
  4000| 6.74e-06  8.88e-05  1.31e-05  3.75e+01  3.75e+01  3.76e-14  5.62e+01 
  4100| 7.50e-06  6.93e-05  1.21e-05  3.75e+01  3.75e+01  3.76e-14  5.82e+01 
  4200| 8.14e-06  4.71e-05  8.60e-06  3.75e+01  3.75e+01  1.46e-13  5.96e+01 
  4300| 8.40e-06  2.28e-05  1.09e-06  3.75e+01  3.75e+01  5.41e-14  6.08e+01 
  4400| 8.20e-06  2.57e-05  1.77e-06  3.75e+01  3.75e+01  5.41e-14  6.27e+01 
  4500| 7.49e-06  4.61e-05  6.67e-06  3.75e+01  3.75e+01  3.76e-14  6.43e+01 
  4600| 6.51e-06  6.49e-05  1.19e-05  3.75e+01  3.75e+01  3.76e-14  6.58e+01 
  4700| 5.49e-06  7.75e-05  1.32e-05  3.75e+01  3.75e+01  3.76e-14  6.74e+01 
  4800| 4.83e-06  8.21e-05  1.32e-05  3.75e+01  3.75e+01  5.41e-14  6.86e+01 
  4900| 4.83e-06  7.83e-05  1.25e-05  3.75e+01  3.75e+01  5.41e-

37.54338872357463

In [24]:
# Huber regression
np.random.seed(hash('huber-regression') % 2 ** 31)

m = 5000
n = 200

x0 = np.random.randn(n)
A = np.random.randn(m, n)
b = A.dot(x0) + 0.01 * np.random.randn(m)
# Add outlier noise.
k = int(0.02 * m)
idx = np.random.randint(m, size=k)
b[idx] += 10 * np.random.randn(k)

x = cvxpy.Variable(n)
prob = cvxpy.Problem(cvxpy.Minimize(cvxpy.sum_entries(cvxpy.huber(A*x - b))))

prob.solve(solver='SCS', verbose=True)
prob.solve(solver='SCS', verbose=True, acceleration_lookback=0)

----------------------------------------------------------------------------
	SCS v2.0.0 - Splitting Conic Solver
	(c) Brendan O'Donoghue, Stanford University, 2012-2017
----------------------------------------------------------------------------
Lin-sys: sparse-indirect, nnz in A = 1045000, CG tol ~ 1/iter^(2.00)
eps = 1.00e-05, alpha = 1.50, max_iters = 5000, normalize = 1, scale = 1.00
acceleration_lookback = 20, rho_x = 1.00e-03
Variables n = 20200, constraints m = 30000
Cones:	primal zero / dual free vars: 5000
	linear vars: 10000
	soc vars: 15000, soc blks: 5000
Setup time: 4.76e-02s
----------------------------------------------------------------------------
 Iter | pri res | dua res | rel gap | pri obj | dua obj | kap/tau | time (s)
----------------------------------------------------------------------------
     0| 1.46e+21  2.29e+21  1.00e+00 -1.88e+25  7.73e+23  4.01e+24  2.20e-01 
   100| 3.89e-03  5.85e-03  1.55e-03  1.49e+03  1.49e+03  4.01e-12  3.06e+00 
   200| 6.90e-04

1487.115339909986